In [1]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer

# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и получить следующий токен через жадную генерацию!

Для загрузки модели и токенайзера вам помогут функции `.from_pretrained`

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [2]:
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name) # Ваш код здесь
tokenizer = AutoTokenizer.from_pretrained(model_name) # ваш код здесь

text = "This is a sample text"

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!
with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt")['input_ids']
    outputs = model(inputs)
    logits = outputs.logits

    next_token_idx: int = torch.argmax(logits[0, -1, :]).item()
    next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"

## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [4]:
text = "This is still a sample text, but"
inputs = tokenizer(text, return_tensors="pt")

results = []
for i in range(10):
    gens = model.generate(
        inputs['input_ids'], 
        attention_mask=inputs['attention_mask'], 
        do_sample=True,
        max_new_tokens=10, 
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.9, 
        top_k=20, 
        repetition_penalty=1.2
    )
    generation: str = tokenizer.decode(gens[0]) # сгенерированный текст
    results.append(generation)

assert len(set(results)) > 1, "Все генерации получились одинаковыми, проверьте опции генерации и флаг do_sample!"

## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side в конструкторе токенизатора.

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left") # ваш код здесь
tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
# Внимание! В данном задании нужна жадная генерация!
# Соберите оба текста в один батч и положите результаты генерации в
# batched_generations
texts = ["This is a sample text", "I'm really tired and this is just about"]
inputs = tokenizer(texts, return_tensors="pt", padding=True)

batched_generations: List[str] = []
generations = model.generate(
    inputs['input_ids'], 
    attention_mask=inputs['attention_mask'], 
    max_new_tokens=10, 
    do_sample=False, # для жадной генерации
    pad_token_id=tokenizer.eos_token_id,
)
batched_generations = tokenizer.batch_decode(generations) # декодирую в строку, потому что в ассерте идет сравнение по "==" и для тензоров оно не работает

# Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации
# в single_generations
single_generations: List[str] = []
for i in range(len(texts)):
    gens = model.generate(
        inputs['input_ids'][i].unsqueeze(0), 
        attention_mask=inputs['attention_mask'][i].unsqueeze(0), 
        max_new_tokens=10, 
        do_sample=False, # для жадной генерации
        pad_token_id=tokenizer.eos_token_id,
    )
    generation: str = tokenizer.decode(gens[0])
    single_generations.append(generation)

In [7]:
assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b

# Скоринг, Perplexity - 10 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [8]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, return_tensors="pt")['input_ids']

Beginning of sentence (BOS) token = `<|endoftext|>`
End of sentence (EOS) token  = `<|endoftext|>`


In [9]:
with torch.no_grad():
    logits = model(inputs).logits # [batch_size=1, seq_len=9, d_model=50257]
    # ваш код здесь!
    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    logits = logits[:, :-1, :]
    # 2. Превращаем logits в log_probs
    log_probs = torch.log_softmax(logits, dim=-1)
    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    # для этого нам поможет torch.gather
    targets = inputs[:, 1:]
    log_probs_next_token = torch.gather(log_probs, dim=-1, index=targets.unsqueeze(-1)) # вероятности для токенов из targets
    # 4. Считаем вероятности и perplexity!
    print("Text probability:", log_probs_next_token.sum().exp().item())
    print("Perplexity:", (-log_probs_next_token.mean()).exp().item())

# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl

Text probability: 2.1782111230277297e-14
Perplexity: 51.0196533203125


# Вопросы - 5 баллов

**Ответьте на вопрсоы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.
2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.



**Ответы:**
1. Близкое к единице: тогда у каждой из условных вероятностей $P(x_i|x_{<i})$ значение будет также ~1 и практически наверняка при генерации будет выбран соответствующий токен.
2. Чем ниже perplexity, тем увереннее модель. В идеале это значение равное 1

# Chat-Models

# Формат - 5 баллов
Как мы обсуждали на лекции, все chat-модели принимают входы в своем особом формате.
Он может быть описан текстом, а может быть заложен в шаблон, который доступен через `tokenizer.apply_chat_template`

In [10]:
device = torch.device("cuda")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct", torch_dtype=torch.half).to(device)
# модель на 15.3 ГБ видеопамяти, там дальше было нелегко

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Давайте посмотрим, как chat модель отработает на обычном тексте. Используйте для генерации сэмплинг и kv cache, выведите 5 результатов генерации.

In [16]:
text = "hello how are you"
inputs = tokenizer(text, return_tensors="pt").to(device)

for i in range(5):
    with torch.no_grad():
        gens = model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'], 
            do_sample=True,
            max_new_tokens=20, 
            pad_token_id=tokenizer.eos_token_id,
        )
        generated_text: str = tokenizer.decode(gens[0]) # сгенерированный текст
        print(generated_text)
        print("====" * 3)

<|begin_of_text|>hello how are you doing today?
I'm doing well, thanks for asking! It's great to connect with you.
<|begin_of_text|>hello how are you?
I'm a bot, so I don't have feelings like humans do, but I'm here
<|begin_of_text|>hello how are you?
I'm doing well, thanks for asking! It's great to hear from you. How about
<|begin_of_text|>hello how are you? i am so excited to be here and to share my story with you. my name is em
<|begin_of_text|>hello how are you?
I'm doing well, thanks for asking! It's great to connect with you. How about


Видим, что текст зачастую выходит мусорный. Это потому что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели.

Не забудьте про опцию add_generation_prefix - она добавляет часть формата, после которой ожидается ответ модели!

In [14]:
messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]

prefix = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# с add_generation_prefix не очень работало, а add_generation_prompt подошел

In [15]:
reference = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

assert prefix.strip() == reference.strip()

Давайте посмотрим, что нам ответит модель!

In [16]:
inputs = tokenizer(prefix, return_tensors="pt").to(device)
with torch.no_grad():
    gens = model.generate(
        inputs['input_ids'], 
        attention_mask=inputs['attention_mask'], 
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text: str = tokenizer.decode(gens[0]) # сгенерированный текст
    print(generated_text)

<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aw, thank you so much! That's very kind of you to say! I'm happy to be here and help you in any way I can.<|eot_id|>


## Benchmark - 15

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа

In [12]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[1]

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

{'question': 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices?',
 'subject': 'global_facts',
 'choices': ['About $300', 'About $3k', 'About $8k', 'About $15k'],
 'answer': 1}

Наша задача здесь решить задачу многоклассовой классификации.
Для этого нужно посчитать
$$P(choices_i | question)$$
т.е. для посчитать вероятность каждого варианта ответа для вопроса. Мы это уже делали кодом выше!

После этого давайте брать самый вероятный ответ и считать, что модель его выбрала.
После этого давайте посчитаем accurracy, т.е. долю правильных ответов.
Вместо вероятностей для подсчета лучше использовать logprobs.

Итого, что нужно сделать:
1. Пройтись по датасету, для каждого question и каждого из соответствующих choices получить самый вероятный ответ.
2. Посчитать итоговый accuracy

**Важно**
1. Выше мы уже написали скоринг текста с помощью LLM, для этого задания можно адаптировать функцию.
2. Если делаете варианты с батчеванием помните: длины choices могут быть разными! Нужно не считать вероятности по паддингам. В этом нам помогут attention_masks из выходов `tokenizer()`
3. В данном задании для простоты мы игнорируем формат ответа llama3 и делаем скоринг по f"{question} {answer}"


Попробуйте для начала написать вариант со скорингом для батча размера 1, а потом для батча размера 3 или 5. Код должен корректно работать для батча любого размера и выдавать одинаковую итоговую точность.

За задание, в котором код работает только с батчом размера 1, 2, 4 можно получить **только 10 баллов**

In [13]:
def sample_to_texts(sample):
    return [sample["question"] + " " + answer for answer in sample["choices"]]

all_samples_formatted = sum([sample_to_texts(sample) for sample in mmlu], [])
print(*all_samples_formatted[2:6], sep="\n")
# ваш код здесь! (чуть ниже на самом деле)

As of 2016, about what percentage of adults aged 18 years or older were overweight? 40%
As of 2016, about what percentage of adults aged 18 years or older were overweight? 80%
What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $300
What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $3k


In [18]:
from tqdm.notebook import tqdm
tokenizer.pad_token = tokenizer.eos_token

max_seq_length = max(len(tokenizer(item)['input_ids']) for item in all_samples_formatted)
max_seq_length

117

In [19]:
def score_batch(questions, choices):
    """
    Функция для подсчета logprobs всех вариантов ответа
    """
    batch_inputs = []
    choices_lens = [] # это для передачи обратно в evaluate_accuracy
    for question, choice_set in zip(questions, choices):
        batch_inputs.extend([f"{question} {choice}" for choice in choice_set])
        choices_lens.append(len(choice_set))

    # здесь все тексты дополнятся паддингами слева до длины "max_seq_length"
    encodings = tokenizer(batch_inputs, padding="max_length", max_length=max_seq_length, truncation=True, return_tensors="pt")
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    # считаю логарифмическую вероятность последнего токена каждого ответа, код очень похож на секцию со скорингом чуть выше
    logprobs_per_answer = []
    for i in range(len(batch_inputs)):
        choice_tokens = input_ids[i, 1:] # это по сути targets
        mask = attention_mask[i, 1:].bool() # маска для учета паддингов
        log_probs_next_token = torch.gather(log_probs[i, :-1, :], dim=-1, index=choice_tokens.unsqueeze(-1))
        valid_log_probs = log_probs_next_token[mask] # тут отбрасываются вероятности для паддинг-токенов
        logprobs_per_answer.append(valid_log_probs.sum().item()) # собираются вероятности ответов

    return logprobs_per_answer, choices_lens

def evaluate_accuracy(batch_size=1):
    correct = 0
    total = 0

    for i in tqdm(range(0, len(mmlu), batch_size)):
        batch = mmlu[i : i + batch_size]

        questions = [item for item in batch['question']]
        choices = [item for item in batch['choices']]
        correct_answers = [item for item in batch['answer']]

        log_probs, choices_lens = score_batch(questions, choices)

        # тут получаются номера предсказаний
        pred_indices = []
        start_idx = 0
        for j in range(batch_size):
            end_idx = start_idx + choices_lens[j] # чтобы ниже отобрать предсказанные вероятности для одного вопроса
            pred_indices.append(torch.tensor(log_probs[start_idx:end_idx]).argmax().item()) # номер выбранного моделью ответа
            start_idx = end_idx

        # для подсчета accuracy
        correct += sum(p == a for p, a in zip(pred_indices, correct_answers))
        total += batch_size

    return correct / total

**Пару слов насчет реализации обработки choices разных длин.** Я пробовал:
- просто не перемножать вероятности для токенов-паддингов при подсчете вероятности текста.
Это не помогает полностью, так как из-за разных длин входного текста софтмакс будет давать разные вероятности и в итоге произведение тоже будет разным для разных batch size
- в логитах заменять соответствующие паддингам значения на -inf, чтобы софтмакс дал на этом месте 0. 
Но он давал NaN
- явно в log_probs проставлять 1 на месте паддинга. Это и не помогло, и к тому же вероятности уже не сложились бы в единицу
- решил при токенизации проставить padding="max_length" и добавлять паддинги до макс. длины (это 102) для каждого текста
Но опять же, надеюсь, что бывает и более лаконичное решение, при котором разные размеры батча не влияли бы на accuracy

In [20]:
accuracy_1 = evaluate_accuracy(batch_size=1)

print(f"Accuracy (batch=1): {accuracy_1:.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy (batch=1): 0.4900


In [17]:
accuracy_2 = evaluate_accuracy(batch_size=2)

print(f"Accuracy (batch=2): {accuracy_2:.4f}")

  0%|          | 0/50 [00:00<?, ?it/s]

Accuracy (batch=2): 0.4900


С учетом классификации на 4+ классов вполне неплохо. А что касается размера батча, то мне не хватало видеопамяти на batch_size > 2. Я тестировал на более простой модели и значение accuracy остается неизменным

# Вопросы - 5 баллов

**Ответьте на следующие вопросы (5 баллов в сумме)**:
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.
2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?


**Ответы:**
1. Суммарная вероятность более длинного ответа может быть чуть ниже за счет произведения большего числа значений из [0, 1]. Поэтому если у правильного ответа будет более длинная формулировка, чем у более коротких, шансы на выбор моделью правильного ответа станут чуть ниже. Из решение самое действенное это, полагаю, использовать логарифмы вероятностей и производить вычисления с ними. Также можно попробовать домножать итоговые вероятности текста на какое-нибудь обратно-пропорциональное его длине значение
2. Я предполагаю, что от добавления меток пользы много не будет, потому что информации в себе они не несут и могут в каком-то смысле запутать модель. Думаю, что ожидать помощи от них можно только если данные, на которых обучалась модель, были похожим образом структурированы